In [2]:
import pandas as pd
from collections import defaultdict, Counter
from tqdm import tqdm
import random

In [3]:
pbp_file = "./play_by_play_2021_22_lite.csv"
pbp = pd.read_csv(pbp_file, encoding='latin-1')


/var/folders/sy/pk0rd4q97v1b5s6drdrrz6q00000gn/T/ipykernel_60275/2907554359.py:2: DtypeWarning: Columns (49,56) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp = pd.read_csv(pbp_file, encoding='latin-1')


In [5]:
pbp_18_19 = pd.read_csv("./play_by_play_2017_18_lite.csv", encoding='latin-1')

/var/folders/sy/pk0rd4q97v1b5s6drdrrz6q00000gn/T/ipykernel_20824/4165166785.py:1: DtypeWarning: Columns (47,54,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_18_19 = pd.read_csv("./play_by_play_2017_18_lite.csv", encoding='latin-1')


In [5]:
pbp.columns.values

array(['xg', 'event_id', 'event_type', 'event', 'secondary_type',
       'event_team', 'event_team_type', 'description', 'period',
       'period_seconds', 'period_seconds_remaining', 'game_seconds',
       'game_seconds_remaining', 'home_score', 'away_score',
       'event_player_1_name', 'event_player_1_type',
       'event_player_2_name', 'event_player_2_type',
       'event_player_3_name', 'event_player_3_type', 'event_goalie_name',
       'strength_state', 'strength_code', 'strength', 'game_winning_goal',
       'empty_net', 'penalty_severity', 'penalty_minutes', 'event_idx',
       'num_on', 'players_on', 'num_off', 'players_off', 'extra_attacker',
       'x', 'y', 'x_fixed', 'y_fixed', 'shot_distance', 'shot_angle',
       'home_skaters', 'away_skaters', 'home_on_1', 'home_on_2',
       'home_on_3', 'home_on_4', 'home_on_5', 'home_on_6', 'home_on_7',
       'away_on_1', 'away_on_2', 'away_on_3', 'away_on_4', 'away_on_5',
       'away_on_6', 'away_on_7', 'home_goalie', 'away_goal

In [1]:
pbp.loc[:, 'event_type'].unique()

NameError: name 'pbp' is not defined

In [ ]:
list(pbp.loc[:, 'away_name'].unique())

In [43]:
# in certain situtations, the captain of an offending team can elect to play 2 men SH
#    for 2m or 1 man SH for 4m.
# in 5/6 scenarios the captain opted for a double minor. Only the Avs (vs ARI 1/14/22)
# elected to go down 2 men for 2m.

cache_4_PIMs = {
    ((240,), ()): 'DBL',
    ((120,120), ()): '2SH',
    ((540,), (300,)): 'DBL',
    ((240, 120), (120,)): 'CPT',
    ((240, 120, 120), (120, 120)): 'CPT',
    ((360, 300), (300, 120)): 'DBL',
    ((360,), (120,)): 'DBL',
    ((120, 120, 120), (120,)): 'DBL',
    ((420, 120), (300,)): '2SH',
    ((300, 120, 120), (300,)): '2SH',
    ((360, 120), (120, 120)): 'CPT',
    ((300, 240), (300,)): 'DBL'
}

def cache_4_min_PIMS(away_pens, home_pens, time, date):
    more_penalized, less_penalized = None, None
    away_more_penalized = sum(away_pens) > sum(home_pens)
    if away_more_penalized:
        more_penalized = away_pens
        less_penalized = home_pens
    else:
        more_penalized = home_pens
        less_penalized = away_pens
    result = cache_4_PIMs[(more_penalized, less_penalized)]
    assert result, f'Coincidental penalty batch not accounted for: {away_pens}, {home_pens}'
    
    if result == 'CPT':
        result = '2SH' if date == '2022-01-14' else 'DBL'
    
    if result == 'DBL':
        pen = Penalty(time, remaining_minors = 1)
        return ([pen], []) if away_more_penalized else ([], [pen])
    else:
        pen1 = Penalty(time)
        pen2 = Penalty(time)
        return ([pen1, pen2], []) if away_more_penalized else ([], [pen1, pen2])



In [9]:
# 7 minute penalty resolver

# in certain situtations, the captain of an offending team can elect to play 2 men SH
#    (1 for 2m, 1 for 5m), or play 1 man SH for 7m

cache_7_PIMs = {
    ((300, 120), ()): '2SH',
    ((540, 300), (420,)): 'CPT',
    ((420,), ()): '1SH',
}

def cache_7_min_PIMS(away_pens, home_pens, time):
    more_penalized, less_penalized = None, None
    away_more_penalized = sum(away_pens) > sum(home_pens)
    if away_more_penalized:
        more_penalized = away_pens
        less_penalized = home_pens
    else:
        more_penalized = home_pens
        less_penalized = away_pens
    result = cache_7_PIMs[(more_penalized, less_penalized)]
    assert result, f'Coincidental penalty batch not accounted for {away_pens}, {home_pens}'
    if result == 'CPT': # CHI @ NSH 12/1/18
        result = '1SH'
            
    if result == '1SH':
        pen = Penalty(time, 300, 'Major', remaining_minors = 1)
        return ([pen], []) if away_more_penalized else ([], [pen])
    else:
        minor = Penalty(time)
        major = Penalty(time, 300, 'Major')
        # since we call .pop() to add penalties to clock, we must ensure minors are 
        # on the clock first (Rule 26.3)
        return ([major, minor], []) if away_more_penalized else ([], [major, minor])
        
    
    

In [9]:
def get_pp_from_penalties(penalties):

    

SyntaxError: incomplete input (75883366.py, line 3)

In [10]:
# this class is used for preprocessing batch penalties
# once the batch of penalties are resolved, the normal Penalty class should be used for the clock
class PrePenalty:
    def __init__(self, time_length = 120):
        self.time_length = time_length
        
# once we process batch penalties, use these objects to add to the penalty clock
class Penalty:
    def __init__(self, time_start, time_length = 120, severity = 'Minor', 
                 is_4v4 = False, remaining_majors = 0, remaining_minors = 0):
#         self.period = period
        self.time_start = time_start
        self.time_length = time_length
        self.severity = severity
        # if True, minor is of the 4v4 type. 
        # both teams down a man; even though this is on the clock, goals don't expire them
        self.is_4v4 = is_4v4
        self.remaining_majors = remaining_majors
        self.remaining_minors = remaining_minors
        # self.double_minor = double_minor
        
        
    def __str__(self):
        period = int(self.time_start / 1200) + 1
        time = self.time_start - (period - 1) * 1200
        return f'{self.severity} Penalty at {time} P{period}'
    
    def __repr__(self):
        period = int(self.time_start / 1200) + 1
        time = self.time_start - (period - 1) * 1200
        return f'{self.severity} Penalty at {time} P{period}'

In [41]:
class Game:
    def __init__(self, away_team, home_team, date, season_type):
        self.away_team = away_team
        self.home_team = home_team
        self.date = date
        self.season_type = season_type # ['R', 'P'], Regular Season or Playoffs
        self.away_penalties_pre = defaultdict(list) # player_name -> [Penalty]
        self.home_penalties_pre = defaultdict(list)
        self.current_time = 0
        self.period = None
        self.power_play = None
        self.strength = (5,5)
        self.strength_to_time = defaultdict(int) # maps strength -> TOI in seconds (tuple -> int)
        # maps team -> {strength -> goals}
        self.strength_to_goals = defaultdict(int)
        self.strength_to_goals[self.away_team] = Counter()
        self.strength_to_goals[self.home_team] = Counter()
        self.strength_to_goals_against = defaultdict(int)
        self.strength_to_goals_against[self.away_team] = Counter()
        self.strength_to_goals_against[self.home_team] = Counter()
        self.away_penalty_clock = []
        self.home_penalty_clock = []
        # penalties that would force a team to have less than 3 skaters on the ice
        # after old ones expire, add queued penalties
        self.queued_away_penalties = [] 
        self.queued_home_penalties = []
        # when penalties expire in OT, teams have an extra skater on the ice until a stoppage
        # i.e. a 4v3 PP ends, play continues as 4v4 until a stoppage
        self.away_OT_extra = 0
        self.home_OT_extra = 0
        # set after stoppages in OT to signify the PP advantage of the home/away team
        # for example if the away team starts OT on the PP, away_OT_PP = 1
        # when the penalty expires, we decrement this and increment both away_OT_extra and home_OT_extra
        self.away_OT_PP = 0
        self.home_OT_PP = 0
        self.regular_season_OT = False
        # for debugging purposes
        self.coincidentals = Counter()
        
    # return the manpower of both teams
    def get_strengths(self):
        if self.period == 4 and self.season_type == 'R': # regular season only
            away_strength = 3 + self.away_OT_extra + self.away_OT_PP
            home_strength = 3 + self.home_OT_extra + self.home_OT_PP

        else: # all other times should be default 5v5 play
            away_strength = 5 - len(self.away_penalty_clock)
            home_strength = 5 - len(self.home_penalty_clock)
        return (away_strength, home_strength)
    
    # takes the period and time of a play and returns the time elapsed since the start of the game
    def get_game_seconds(self, period, time):
        return (period - 1) * 1200 + time

    # bulky function that looks at both away/home penalty clocks and removes expired ones
    # if penalties expire and there are queued penalties, we should add them to the clock here
    def update_penalty_clock(self, period, period_seconds, debug_mode = False):
        # get time elapsed since last update
        old_time = self.current_time
        current_seconds = self.get_game_seconds(period, period_seconds)
        secs_elapsed = current_seconds - old_time
        
        # update current time
        self.current_time = current_seconds # period_seconds
        self.period = period
        
        old_strengths = self.get_strengths()
        
        # update away penalty clock
        away_expired_penalties = []
        new_away_penalty_clock = []
        for pen in self.away_penalty_clock:
            # curr_expiration = (pen.period - 1) * 1200 + pen.time_start + pen.time_length
            curr_expiration = pen.time_start + pen.time_length
            if curr_expiration > self.current_time:
                new_away_penalty_clock.append(pen)
            else: # if current penalty expired, see if any are remaining
                if pen.remaining_majors:
                    new_pen_start = min(pen.time_start + 300, self.current_time)
                    rem_majors = pen.remaining_majors - 1
                    new_pen = Penalty(new_pen_start, 500, 'Major', 
                                      remaining_majors = rem_majors, remaining_minors = pen.remaining_minors)
                    new_away_penalty_clock.append(new_pen)
                elif pen.remaining_minors:
                    new_pen_start = min(pen.time_start + 120, self.current_time)
                    rem_minors = pen.remaining_minors - 1
                    new_pen = Penalty(new_pen_start, remaining_minors = rem_minors)
                    new_away_penalty_clock.append(new_pen)
                else: # keep the expired penalties in case there are queued penalties and we can adjust time_start accordingly
                    away_expired_penalties.append(pen)
                    
        away_num_added = 0 # amount of penalties that were queued and have been added to the clock
        # if there is room for another penalty, we should check if there are queued penalties and add them if possible
        if len(new_away_penalty_clock) == 1: 
            if away_expired_penalties and self.queued_away_penalties: # if one expired and one is on deck, add one
                first_expired = min(away_expired_penalties, key = lambda pen: pen.time_start + pen.time_length)
                new_pen = self.queued_away_penalties.pop()
                new_pen.time_start = first_expired.time_start + first_expired.time_length
                new_away_penalty_clock.append(new_pen)
                away_num_added = 1
        elif len(new_away_penalty_clock) == 0:
            if away_expired_penalties and self.queued_away_penalties:
                if len(self.queued_away_penalties) == 2: # if there are 2 queued penalties, we know that 2 just expired
                    exp_pen_1 = away_expired_penalties[0]
                    new_pen_1 = self.queued_away_penalties.pop()
                    new_pen_1.time_start = exp_pen_1.time_start + exp_pen_1.time_length
                    new_away_penalty_clock.append(new_pen_1)
                    
                    exp_pen_2 = away_expired_penalties[1]
                    new_pen_2 = self.queued_away_penalties.pop()
                    new_pen_2.time_start = exp_pen_2.time_start + exp_pen_2.time_length
                    new_away_penalty_clock.append(new_pen_2)
                    away_num_added = 2
                elif len(self.queued_away_penalties) == 1:
                    first_expired = min(away_expired_penalties, key = lambda pen: pen.time_start + pen.time_length)
                    new_pen = self.queued_away_penalties.pop()
                    new_pen.time_start = first_expired.time_start + first_expired.time_length
                    new_away_penalty_clock.append(new_pen)
                    away_num_added = 1
        
        num_expired = len(self.away_penalty_clock) - len(new_away_penalty_clock)
        if num_expired and self.regular_season_OT:
            self.away_OT_extra += num_expired
            self.home_OT_extra += num_expired
            self.home_OT_PP -= num_expired
        
        self.away_penalty_clock = new_away_penalty_clock
        away_expired_penalties = away_expired_penalties[away_num_added:] # for the strength metrics
        
        # update home penalty clock
        home_expired_penalties = []
        new_home_penalty_clock = []
        for pen in self.home_penalty_clock:
            # curr_expiration = (pen.period - 1) * 1200 + pen.time_start + pen.time_length
            curr_expiration = pen.time_start + pen.time_length
            if curr_expiration > self.current_time:
                new_home_penalty_clock.append(pen)    
            else: # if current penalty expired, see if any are remaining or queue new penalties
                if pen.remaining_majors:
                    new_pen_start = min(pen.time_start + 300, self.current_time)
                    rem_majors = pen.remaining_majors - 1
                    new_pen = Penalty(new_pen_start, 500, 'Major', 
                                      remaining_majors = rem_majors, remaining_minors = pen.remaining_minors)
                    new_home_penalty_clock.append(new_pen)
                elif pen.remaining_minors:
                    new_pen_start = min(pen.time_start + 120, self.current_time)
                    rem_minors = pen.remaining_minors - 1
                    new_pen = Penalty(new_pen_start, remaining_minors = rem_minors)
                    new_home_penalty_clock.append(new_pen)
                else: # keep the expired penalties in case there are queued penalties and we can adjust time_start accordingly
                    home_expired_penalties.append(pen)
          
        home_num_added = 0 # number of queued penalties that were added to the clock
        # if there is room for another penalty, we should check if there are queued penalties and add them if possible
        if len(new_home_penalty_clock) == 1: 
            if home_expired_penalties and self.queued_home_penalties: # if one expired and one is on deck, add one
                first_expired = min(home_expired_penalties, key = lambda pen: pen.time_start + pen.time_length)
                new_pen = self.queued_home_penalties.pop()
                new_pen.time_start = first_expired.time_start + first_expired.time_length
                new_home_penalty_clock.append(new_pen)
                home_num_added = 1
        elif len(new_home_penalty_clock) == 0:
            if home_expired_penalties and self.queued_home_penalties:
                if len(self.queued_home_penalties) == 2: # if there are 2 queued penalties, we know that 2 just expired
                    exp_pen_1 = home_expired_penalties[0]
                    new_pen_1 = self.queued_home_penalties.pop()
                    new_pen_1.time_start = exp_pen_1.time_start + exp_pen_1.time_length
                    new_home_penalty_clock.append(new_pen_1)
                    
                    exp_pen_2 = home_expired_penalties[1]
                    new_pen_2 = self.queued_home_penalties.pop()
                    new_pen_2.time_start = exp_pen_2.time_start + exp_pen_2.time_length
                    new_home_penalty_clock.append(new_pen_2)
                    home_num_added = 2
                elif len(self.queued_home_penalties) == 1:
                    first_expired = min(home_expired_penalties, key = lambda pen: pen.time_start + pen.time_length)
                    new_pen = self.queued_home_penalties.pop()
                    new_pen.time_start = first_expired.time_start + first_expired.time_length
                    new_home_penalty_clock.append(new_pen)
                    home_num_added = 1
                    
        num_expired = len(self.home_penalty_clock) - len(new_home_penalty_clock)
        if num_expired and self.regular_season_OT:
            self.home_OT_extra += num_expired
            self.away_OT_extra += num_expired
            self.away_OT_PP -= num_expired
                    
        self.home_penalty_clock = new_home_penalty_clock
        home_expired_penalties = home_expired_penalties[home_num_added:]
        
        self.strength = self.get_strengths()
        
        if home_num_added or away_num_added:
            print(f'{self.away_team} @ {self.home_team} {self.date}')
        
        # exclude shootouts, which mess up the data
        if not (self.season_type == 'R' and self.period > 4): 
            self.update_strength_metrics(away_expired_penalties, home_expired_penalties, old_strengths, secs_elapsed, debug_mode)
            
        if debug_mode:
            print(f'Updating Penalty Clock at P:{period} {period_seconds}s')
            print(f'Away: {self.away_penalty_clock}')
            print(f'Home: {self.home_penalty_clock}')
            print(self.get_strengths())
            
    # update our strength tracking metrics by looking at expired penalties
    # if pens expired here, we know no goal was scored (unless major/match)
    def update_strength_metrics(self, away_expired_penalties, home_expired_penalties, old_strengths, secs_elapsed, debug_mode):
        away_expired = len(away_expired_penalties)
        home_expired = len(home_expired_penalties)
        if away_expired + home_expired == 1:
            if away_expired:
                pen = away_expired_penalties[0]
            else:
                pen = home_expired_penalties[0]
            old_strength_time = self.current_time - pen.time_length - pen.time_start
            self.strength_to_time[old_strengths] += secs_elapsed - old_strength_time
            self.strength_to_time[self.strength] += old_strength_time
        elif not(away_expired or home_expired):
            self.strength_to_time[old_strengths] += secs_elapsed
        else: # complex case
            # maps expiration_time -> <'away'/'home'> to indicate who the expiring penalty was on
            if len(away_expired_penalties) + len(home_expired_penalties) > 2:
                print(f'multiple expiring penalties: {self.away_team} @ {self.home_team} {self.date}')
            expired_pens = defaultdict(list) 
            expiration_times = set([]) # stores unique penalty expiration times
            
            for away_pen in away_expired_penalties:
                expiration = away_pen.time_start + away_pen.time_length
                expiration_times.add(expiration)
                expired_pens[expiration].append('away')
                
            for home_pen in home_expired_penalties:
                expiration = home_pen.time_start + home_pen.time_length
                expiration_times.add(expiration)
                expired_pens[expiration].append('home')
                
            # sort in ascending order of expiration
            expiration_times = sorted(list(expiration_times))
            
            new_strengths = old_strengths
            # keeps track of time in the past as we are retroactively inferring TOI for each manpower
            checkpoint = self.current_time - secs_elapsed
            for curr_exp_time in expiration_times:
                curr_expired_pens = expired_pens[curr_exp_time]
                time_diff = curr_exp_time - checkpoint # time that given manpower was on ice
                
                self.strength_to_time[new_strengths] += time_diff
                checkpoint += time_diff
                # for each penalty that expires at exactly this time, increment the manpower
                for team in curr_expired_pens:
                    if team == 'away':
                        new_strengths = (new_strengths[0] + 1, new_strengths[1])
                    else:
                        new_strengths = (new_strengths[0], new_strengths[1] + 1)
                        
            # don't forget to increment the current manpower TOI with the remaining time
            self.strength_to_time[new_strengths] += self.current_time - checkpoint
        if debug_mode:
            print(self.strength_to_time)
            
    def update_strength(self):
        self.strength = self.get_strengths()     
    
    # this helper function is called when a goal is scored to remove the first (non-4v4) minor to expire
    def get_expiring_penalty(self, team_penalty_clock):
        first_expiration = float('inf')
        first_expiration_pen = None
        for i, pen in enumerate(team_penalty_clock):
            # curr_expiration = (pen.period - 1) * 1200 + pen.time_start + pen.time_length
            curr_expiration = pen.time_start + pen.time_length
            if pen.severity == 'Minor' and not pen.is_4v4 and curr_expiration < first_expiration:
                first_expiration_pen = pen
                first_expiration = curr_expiration
        return first_expiration_pen
    
    # - helper function called after a power play goal
    # - we look at the short-handed team's penalty clock and remove the first non-coincidental minor set to expire
    # - additionally, if we remove a minor penalty and there are queued penalties, we should add them here until there
    #     are 2 penalties on the clock, or there are no queued penalties.
    # - retval: whether or not a minor penalty was removed
    def remove_minor(self, team):
        if team == self.away_team: 
            expiring_penalty = self.get_expiring_penalty(self.away_penalty_clock)
            if expiring_penalty:
                self.away_penalty_clock.remove(expiring_penalty)
                if expiring_penalty.remaining_minors > 0:
                    rem_minors = expiring_penalty.remaining_minors - 1
                    new_pen = Penalty(self.current_time, remaining_minors = rem_minors)
                    self.away_penalty_clock.append(new_pen)
                elif self.queued_away_penalties: # check for queued penalties
                    new_pen = self.queued_away_penalties.pop()
                    new_pen.time_start = self.current_time
                    self.away_penalty_clock.append(new_pen)
        else:
            expiring_penalty = self.get_expiring_penalty(self.home_penalty_clock)
            if expiring_penalty:
                self.home_penalty_clock.remove(expiring_penalty)
                if expiring_penalty.remaining_minors > 0:
                    rem_minors = expiring_penalty.remaining_minors - 1
                    new_pen = Penalty(self.current_time, remaining_minors = rem_minors)
                    self.home_penalty_clock.append(new_pen)
                elif self.queued_home_penalties: # check for queued penalties
                    new_pen = self.queued_home_penalties.pop()
                    new_pen.time_start = self.current_time
                    self.home_penalty_clock.append(new_pen)
            
    # - this function is called when a goal is scored
    # - we first call update_penalty_clock() to remove penalties that may have expired due to time
    # - we then consider the on-ice manpower and if the scoring team was on the PP, remove a minor 
    #     penalty from the other team if possible.
    def add_goal(self, play, debug_mode = False):
        period_goal_scored = play['period']
        time_goal_scored = play['period_seconds']
        
        # remove penalties that have expired
        self.update_penalty_clock(period_goal_scored, time_goal_scored, debug_mode)
        
        away_strength, home_strength = self.get_strengths()
        if debug_mode:
            print(f'GOAL!! {away_strength}v{home_strength}')
            
        # Rule 16.2: no penalty shall expire due to a penalty shot goal
        if play['secondary_type'] == 'Penalty Shot':
            return
        
        # add goals to the given strength in self.strength_to_goals
        self.update_goal_metric(play)
        
        if away_strength != home_strength: # no penalty shall expire at even strength
            # remove any applicable minor penalties on the clock, iff scoring team was on PP
            team_scored = play['event_team']
            if team_scored == self.away_team and away_strength > home_strength:
                self.remove_minor(self.home_team)
            elif team_scored == self.home_team and home_strength > away_strength:
                self.remove_minor(self.away_team)
            
        self.update_strength()
        
        if debug_mode:
            print('After Goal:')
            print(f'Away: {self.away_penalty_clock}')
            print(f'Home: {self.home_penalty_clock}')
    
    
    def update_goal_metric(self, play):
        scoring_team = play['event_team']
        away_strength, home_strength = self.get_strengths()
        if scoring_team == self.away_team:
            if self.get_strengths() == (3, 5):
                print(f'3 on 5 goal: {self.away_team} @ {self.home_team} {self.date}')
            self.strength_to_goals[scoring_team][(away_strength, home_strength)] += 1
            self.strength_to_goals_against[self.home_team][(home_strength, away_strength)] += 1
        elif scoring_team == self.home_team:
            if self.get_strengths() == (5, 3):
                print(f'3 on 5 goal: {self.away_team} @ {self.home_team} {self.date}')
            self.strength_to_goals[scoring_team][(home_strength, away_strength)] += 1
            self.strength_to_goals_against[self.away_team][(away_strength, home_strength)] += 1

    # add a penalty to the current batch of penalties
    # we use the PrePenalty class as these are being preprocessed. we really only care about the 
    # possibility of 5+2s, 2+2s, etc, and the total PIMs
    def add_penalty(self, play):
        penalized_team = play['event_team']
        severity = play['penalty_severity']
            
        penalty = PrePenalty()
        if severity == 'Minor': # check PIMs if 2 or 4
            if play['penalty_minutes'] == 4: # double minor is 2 minors back to back, not a single 4 minute block
                penalty.time_length = 240
            elif play['penalty_minutes'] == 6: # triple minor
                penalty.time_length = 360
        elif severity == 'Major' or severity == 'Match': # 5 minutes
            penalty.time_length = 300

        penalized_player = play['event_player_1_name']
        if penalized_team == self.away_team:
            self.away_penalties_pre[penalized_player].append(penalty)
        else:
            self.home_penalties_pre[penalized_player].append(penalty)

    # once the current batch of penalties is announced, determine the on ice strength
    # using self.<away/home>_penalty_clock, try to resolve coincidental penalties
    # once we have coincidental penalties, only store the penalties on the clock
    # if the penalty clock is full, add the rest of them to self.queued_<away/home>_penalties
    def process_penalties(self, play, debug_mode = False):
        # group penalties by player. if they have multiple, add the PIMs and we can infer them
        away_pens = self.get_team_pens(self.away_penalties_pre)
        home_pens = self.get_team_pens(self.home_penalties_pre)
        
        # clear expired penalties and keep self.current_time updated
        self.update_penalty_clock(play['period'], play['period_seconds'])
        
        away_PIMs = sum(away_pens)
        home_PIMs = sum(home_pens)
        PIMs_diff = away_PIMs - home_PIMs
        abs_PIMs_diff = abs(PIMs_diff)
        
        away_team_penalized = PIMs_diff > 0
        penalized_team = self.away_team if away_team_penalized else self.home_team
        
        # we keep temporary lists of the penalties to be added in case we need to queue one or more
        # i.e. if a team has 1 on the clock and takes 2 more penalties, we can only add 1 and need to queue the other
        away_temp_pens = []
        home_temp_pens = []
        
        match abs_PIMs_diff:
            case 120: # 2m PP
                pen = Penalty(self.current_time)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 300: # 5m PP
                pen = Penalty(self.current_time, 300, 'Major')
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 0: # Full strength unless both teams have exactly 1 2m minor
                if away_pens == (120,) and home_pens == (120,) and self.get_strengths() == (5,5):
                    pen1 = Penalty(self.current_time, is_4v4 = True)
                    pen2 = Penalty(self.current_time, is_4v4 = True)
                    away_temp_pens.append(pen1)
                    home_temp_pens.append(pen2)
            case 240:
                away_temp_pens, home_temp_pens = cache_4_min_PIMS(away_pens, home_pens, self.current_time, play['game_date'])
            case 60: # 5 vs 2, 2
                print('PIM difference == 1')
                print('Assuming 1 major vs 2 minors')
                print(away_pens, home_pens)
                minor1 = Penalty(self.current_time)
                minor2 = Penalty(self.current_time)
                major = Penalty(self.current_time, 500, 'Major')
                if self.current_time >= 3300:
                    print(f'L5M/OT: {self.away_pens}, {self.home_pens}')
                    assert False, f"L5M/OT situation {self.away_team} @ {self.home_team} {self.date}"
                    major = Penalty(self.current_time, 300, 'Major')
                else:
                    if away_team_penalized: # assign the major to the away team
                        away_temp_pens.append(major)
                        home_temp_pens = [minor1, minor2]
                    else: # assign the major to the home team
                        away_temp_pens = [minor1, minor2]
                        home_temp_pens.append(major)
            case 180: # 5 vs 2
                # rule 19.4 for last 5 mins and overtime
                if self.current_time >= 3300:
                    print(f'L5M/OT: {away_pens}, {home_pens}')
                    print(f'{self.away_team} @ {self.home_team} {self.date}')
                    major = Penalty(self.current_time, 180, 'Major')
                    away_temp_pens.append(major) if away_team_penalized else home_temp_pens.append(major)
                else:
                    major = Penalty(self.current_time, 300, 'Major')
                    minor = Penalty(self.current_time)
                    if away_team_penalized: # assign the major to the away team
                        away_temp_pens.append(major)
                        home_temp_pens.append(minor)
                    else: # assign the major to the home team
                        away_temp_pens.append(minor)
                        home_temp_pens.append(major)
            case 360: # 2 + 2 + 2
                print('PIM difference == 6')
                print('Assuming a 2+2+2')
                print(f'{self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                pen = Penalty(self.current_time, remaining_minors = 2)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 420: # 5 + 2
                print('PIM difference == 7')
                print(f'{self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                away_temp_pens, home_temp_pens = cache_7_min_PIMS(away_pens, home_pens, self.current_time)
            case 540: # 5 + 2 + 2
                print('PIM difference == 9')
                print('Assuming a 5+2+2')
                print(f'{self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                pen = Penalty(self.current_time, 300, 'Major', remaining_minors = 2)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case 600: # 5 + 5, 5 // 5
                print('PIM difference == 9')
                print('Assuming a 5+5')
                print(f'{self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                pen = Penalty(self.current_time, 300, 'Major', remaining_majors = 1)
                away_temp_pens.append(pen) if away_team_penalized else home_temp_pens.append(pen)
            case default: # disregard for now
                print(abs_PIMs_diff)
                print('Default case for PIM matching')
                print(f'{self.away_team} @ {self.home_team} {self.date}')
                print(away_pens, home_pens)
                assert False, "Current batch of coincidental penalties not accounted for: {away_pens}, {home_pens}"
                
        self.add_penalties_to_clock(away_temp_pens, home_temp_pens)
        
        if self.regular_season_OT:
            self.reset_OT_strengths()
            
        self.update_strength()

        # reset pre-processed penalties
        self.away_penalties_pre = defaultdict(list)
        self.home_penalties_pre = defaultdict(list)
                
        if debug_mode:
            print('Processed penalties. Resulting Penalty Clock: ')
            print(f'PIMs_diff: {PIMs_diff}')
            print(f'Away: {self.away_penalty_clock}')
            print(f'Home: {self.home_penalty_clock}')
        
    # helper function that takes all penalties drawn and attempts to add them to the penalty clock
    # the penalty clock cannot have more than 2 penalties, so we add the rest to the queue 
    # the queued penalties will begin once earlier ones expire
    def add_penalties_to_clock(self, temp_away_pens, temp_home_pens):
        # see if we can add these penalties to the clock, else, queue them
        if len(self.away_penalty_clock) == 2:
            self.queued_away_penalties += temp_away_pens
        elif len(self.away_penalty_clock) == 1:
            if temp_away_pens:
                pen = temp_away_pens.pop()
                self.away_penalty_clock.append(pen)
                self.queued_away_penalties += temp_away_pens
        elif len(self.away_penalty_clock) == 0:
            if len(temp_away_pens) > 2:
                pen1 = temp_away_pens.pop()
                pen2 = temp_away_pens.pop()
                self.away_penalty_clock = [pen1, pen2]
                self.queued_away_penalties += temp_away_pens
            else:
                self.away_penalty_clock = temp_away_pens
        else:
            assert "Away Penalty Clock is misconfigured"
            
        # add home_penalties now
        if len(self.home_penalty_clock) == 2:
            self.queued_home_penalties += temp_home_pens
        elif len(self.home_penalty_clock) == 1:
            if temp_home_pens:
                pen = temp_home_pens.pop()
                self.home_penalty_clock.append(pen)
                self.queued_home_penalties += temp_home_pens
        elif len(self.home_penalty_clock) == 0:
            if len(temp_home_pens) > 2:
                pen1 = temp_home_pens.pop()
                pen2 = temp_home_pens.pop()
                self.home_penalty_clock = [pen1, pen2]
                self.queued_home_penalties += temp_home_pens
            else:
                self.home_penalty_clock = temp_home_pens
        else:
            assert "Home Penalty Clock is misconfigured"
    
    # team_penalties: player -> [Penalty]
    # groups PIMs by player
    # retval: tuple of each players total PIMs
    def get_team_pens(self, team_penalties):
        team_pens = ()
        for player, pens in team_penalties.items():
            curr_PIMs = sum(pen.time_length for pen in pens)
            team_pens = (*team_pens, curr_PIMs)
        return tuple(sorted(list(team_pens), reverse = True))
    
    # clear 4v4 penalties
    def start_OT(self):
        self.regular_season_OT = True
        self.away_penalty_clock = [pen for pen in self.away_penalty_clock if not pen.is_4v4]
        assert len(self.queued_away_penalties) == 0, f"There are queued penalties heading to OT {self.away_team} v {self.home_team} {self.date}"
        self.home_penalty_clock = [pen for pen in self.home_penalty_clock if not pen.is_4v4]
        assert len(self.queued_home_penalties) == 0, f"There are queued penalties heading to OT {self.away_team} v {self.home_team} {self.date}"
        self.reset_OT_strengths()
        
    # after any stoppage in regular season OT, reset back to the default 3v3
    # or set PP fields if there is a power play
    def reset_OT_strengths(self):
        self.away_OT_extra = 0
        self.home_OT_extra = 0
        self.away_OT_PP = 0
        self.home_OT_PP = 0
        OT_PP = len(self.away_penalty_clock) - len(self.home_penalty_clock)
        if OT_PP > 0:
            self.home_OT_PP = OT_PP
        elif OT_PP < 0:
            self.away_OT_PP = -OT_PP
    
    # debugging function
    def process_penalties_debug(self):
        away_pens = self.get_team_pens(self.away_penalties_pre)
        home_pens = self.get_team_pens(self.home_penalties_pre)

        self.coincidentals[(away_pens, home_pens)] += 1
        self.coincidentals[(home_pens, away_pens)] += 1
        self.away_penalties_pre = defaultdict(list)
        self.home_penalties_pre = defaultdict(list)
        
    # designed to be used with the coincidental penalty debugging function
    def add_penalty_debug(self, play):
        penalized_team = play['event_team']
        severity = play['penalty_severity']
        period = play['period']
        time = play['period_seconds']
        self.current_time = time
        self.period = period

        penalty = PrePenalty()
        if severity == 'Minor': # check PIMs if 2 or 4
            PIMs = play['penalty_minutes']
            if PIMs == 4: # double minor is 2 minors back to back, not a single 4 minute block
                penalty.time_length = 240
            elif PIMs == 6: 
                penalty.time_length = 360
                penalty.severity = 'Triple Minor'
        elif severity == 'Major' or severity == 'Match': # 5 minutes
            penalty.time_length = 300

        penalized_player = play['event_player_1_name']
        if penalized_team == self.away_team:
            self.away_penalties_pre[penalized_player].append(penalty)
        else:
            self.home_penalties_pre[penalized_player].append(penalty)

In [38]:
def scrape_pbp(playoffs = False):
    teams = ['Pittsburgh Penguins',
     'Seattle Kraken',
     'Montréal Canadiens',
     'New York Rangers',
     'Chicago Blackhawks',
     'Vancouver Canucks',
     'Toronto Maple Leafs',
     'Tampa Bay Lightning',
     'Dallas Stars',
     'New York Islanders',
     'Arizona Coyotes',
     'Winnipeg Jets',
     'Vegas Golden Knights',
     'Minnesota Wild',
     'Ottawa Senators',
     'Carolina Hurricanes',
     'St. Louis Blues',
     'Calgary Flames',
     'Anaheim Ducks',
     'San Jose Sharks',
     'Columbus Blue Jackets',
     'Florida Panthers',
     'Colorado Avalanche',
     'Los Angeles Kings',
     'Boston Bruins',
     'Washington Capitals',
     'Edmonton Oilers',
     'Detroit Red Wings',
     'Buffalo Sabres',
     'Nashville Predators',
     'Philadelphia Flyers',
     'New Jersey Devils']

    overall_strength_to_TOI = {}
    overall_strength_to_goals = {}
    overall_strength_to_goals_against = {}

    for year in range(16, 22):
        season = int(f'20{year}')
        overall_strength_to_TOI[season] = {}
        overall_strength_to_goals[season] = {}
        overall_strength_to_goals_against[season] = {}
        for _team in teams:
            overall_strength_to_TOI[season][_team] = Counter()
            overall_strength_to_goals[season][_team] = Counter()
            overall_strength_to_goals_against[season][_team] = Counter()
    
    current_game = None
    penalty_just_called = False
    debug = False

    # these are irrelevant to on-ice manpower and PP goals (misconducts usually paired with a minor/major)
    irrelevant_penalties = set(['Penalty Shot', 'Misconduct', 'Game Misconduct'])
    for year in range(16, 22):
        file = f'./play_by_play_20{year}_{year+1}_lite.csv'
        curr_pbp = pd.read_csv(file, encoding='latin-1')
        for _, play in tqdm(curr_pbp.iterrows()):
            if playoffs and play['season_type'] == 'R':
                continue
            else:
                event = play['event_type']

                away_team, home_team, date = play['away_name'], play['home_name'], play['game_date']
                if current_game: # sometimes, games don't have the 'GAME_SCHEDULED' event, need to manually check
                    if away_team != current_game.away_team or home_team != current_game.home_team or date != current_game.date:
                        overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.away_team, False)
                        overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.home_team, True)
                        current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])

                if penalty_just_called and event != 'PENALTY': # once all penalties are announced, process them and figure out the manpower
                    current_game.process_penalties(play, debug)
                    penalty_just_called = False

                if play['period'] == 4 and current_game.season_type == 'R':
                    if not current_game.regular_season_OT:
                        current_game.start_OT()
                    elif play['event_type'] in set(['STOP', 'CHALLENGE', 'PENALTY']):
                        current_game.reset_OT_strengths()

                match event:
                    case 'GAME_SCHEDULED':
                        if current_game:
                            overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.away_team, False)
                            overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.home_team, True)
                        current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])
                    case 'PENALTY':
                        if play['penalty_severity'] not in irrelevant_penalties:  # minor/major/misconduct/game misconduct/match
                            current_game.add_penalty(play)
                            penalty_just_called = True
                    case 'GOAL': # exclude penalty shots and shootout goals
                        if play['secondary_type'] != 'Penalty Shot' and play['period'] != 5: # exclude SO
                            current_game.add_goal(play, debug)
                    case default:
                        current_game.update_penalty_clock(play['period'], play['period_seconds'], debug)


        print('Done')
    return overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against

In [47]:
def scrape_pbp_22(playoffs = False):
    teams = ['Pittsburgh Penguins',
     'Seattle Kraken',
     'Montréal Canadiens',
     'New York Rangers',
     'Chicago Blackhawks',
     'Vancouver Canucks',
     'Toronto Maple Leafs',
     'Tampa Bay Lightning',
     'Dallas Stars',
     'New York Islanders',
     'Arizona Coyotes',
     'Winnipeg Jets',
     'Vegas Golden Knights',
     'Minnesota Wild',
     'Ottawa Senators',
     'Carolina Hurricanes',
     'St. Louis Blues',
     'Calgary Flames',
     'Anaheim Ducks',
     'San Jose Sharks',
     'Columbus Blue Jackets',
     'Florida Panthers',
     'Colorado Avalanche',
     'Los Angeles Kings',
     'Boston Bruins',
     'Washington Capitals',
     'Edmonton Oilers',
     'Detroit Red Wings',
     'Buffalo Sabres',
     'Nashville Predators',
     'Philadelphia Flyers',
     'New Jersey Devils']

    overall_strength_to_TOI = {}
    overall_strength_to_goals = {}
    overall_strength_to_goals_against = {}

    for year in range(22, 23):
        season = int(f'20{year}')
        overall_strength_to_TOI[season] = {}
        overall_strength_to_goals[season] = {}
        overall_strength_to_goals_against[season] = {}
        for _team in teams:
            overall_strength_to_TOI[season][_team] = Counter()
            overall_strength_to_goals[season][_team] = Counter()
            overall_strength_to_goals_against[season][_team] = Counter()
    
    current_game = None
    penalty_just_called = False
    debug = False

    # these are irrelevant to on-ice manpower and PP goals (misconducts usually paired with a minor/major)
    irrelevant_penalties = set(['Penalty Shot', 'Misconduct', 'Game Misconduct'])
    for year in range(22, 23):
        file = f'./play_by_play_20{year}_{year+1}_lite.csv'
        curr_pbp = pd.read_csv(file)
        for _, play in tqdm(curr_pbp.iterrows()):
            if playoffs and play['season_type'] == 'R':
                continue
            else:
                event = play['event_type']

                away_team, home_team, date = play['away_name'], play['home_name'], play['game_date']
                if current_game: # sometimes, games don't have the 'GAME_SCHEDULED' event, need to manually check
                    if away_team != current_game.away_team or home_team != current_game.home_team or date != current_game.date:
                        overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.away_team, False)
                        overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.home_team, True)
                        current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])

                if penalty_just_called and event != 'PENALTY': # once all penalties are announced, process them and figure out the manpower
                    current_game.process_penalties(play, debug)
                    penalty_just_called = False

                if play['period'] == 4 and current_game.season_type == 'R':
                    if not current_game.regular_season_OT:
                        current_game.start_OT()
                    elif play['event_type'] in set(['STOP', 'CHALLENGE', 'PENALTY']):
                        current_game.reset_OT_strengths()

                match event:
                    case 'GAME_SCHEDULED':
                        if current_game:
                            overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.away_team, False)
                            overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, current_game.strength_to_time, current_game.strength_to_goals, current_game.strength_to_goals_against, int(f'20{year}'), current_game.home_team, True)
                        current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])
                    case 'PENALTY':
                        if play['penalty_severity'] not in irrelevant_penalties:  # minor/major/misconduct/game misconduct/match
                            current_game.add_penalty(play)
                            penalty_just_called = True
                    case 'GOAL': # exclude penalty shots and shootout goals
                        if play['secondary_type'] != 'Penalty Shot' and play['period'] != 5: # exclude SO
                            current_game.add_goal(play, debug)
                    case default:
                        current_game.update_penalty_clock(play['period'], play['period_seconds'], debug)


        print('Done')
    return overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against

In [39]:
def update_team_statistics(overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against, strength_to_time, strength_to_goals, strength_to_goals_against, year, team, is_home):
    if team in ['Canadian All-Stars', 'American All-Stars']:
        return overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against
    else:
        if is_home:       
            for _strength, _time in strength_to_time.items():
                home_strength = (_strength[1], _strength[0])
                overall_strength_to_TOI[year][team][home_strength] += _time
            overall_strength_to_goals[year][team] += strength_to_goals[team]
            overall_strength_to_goals_against[year][team] += strength_to_goals_against[team]
        else:
            overall_strength_to_TOI[year][team] += strength_to_time
            overall_strength_to_goals[year][team] += strength_to_goals[team]
            overall_strength_to_goals_against[year][team] += strength_to_goals_against[team]
        return overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against

    
    

In [50]:
def get_2022_data():
    # 2022-23 season
    overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = scrape_pbp_22()

    strengths = [(5,5), (5,4), (4,5), (4,4), (3,3), (4,3), (3,4), (5,3), (3,5)]
    columns = ['season', 'team', 'strength', 'TOI', 'goals', 'goals_against']
    data = []
    for year in range(2022, 2023):
        teams = overall_strength_to_TOI[year].keys()
        for _team in teams:
            curr_row = [year, _team]
            for _strength in strengths:
                curr_toi = overall_strength_to_TOI[year][_team][_strength]
                curr_goals = overall_strength_to_goals[year][_team][_strength]
                curr_goals_against = overall_strength_to_goals_against[year][_team][_strength]
                new_row = curr_row + [_strength, curr_toi, curr_goals, curr_goals_against]
                data.append(new_row)

    df = pd.DataFrame(data, columns=columns)
    df.to_csv("./nhl_strength_data_22.csv")

get_2022_data()


/var/folders/sy/pk0rd4q97v1b5s6drdrrz6q00000gn/T/ipykernel_20824/262461324.py:57: DtypeWarning: Columns (49,56) have mixed types. Specify dtype option on import or set low_memory=False.
  curr_pbp = pd.read_csv(file)
48323it [00:04, 14602.71it/s]

3 on 5 goal: Ottawa Senators @ Tampa Bay Lightning 2022-11-01


60098it [00:05, 13625.99it/s]

PIM difference == 1
Assuming 1 major vs 2 minors
(120, 120) (300,)


74785it [00:06, 14086.71it/s]

...

Carolina Hurricanes @ New Jersey Devils 2023-01-01


222115it [00:16, 14507.28it/s]

multiple expiring penalties: Florida Panthers @ Montréal Canadiens 2023-01-19


310585it [00:22, 13806.71it/s]


Done


In [37]:
overall_strength_to_TOI, overall_strength_to_goals, overall_strength_to_goals_against = scrape_pbp(playoffs=True)

strengths = [(5,5), (5,4), (4,5), (4,4), (3,3), (4,3), (3,4), (5,3), (3,5)]
columns = ['season', 'team', 'strength', 'TOI', 'goals', 'goals_against']
data = []
for year in range(2016, 2022):
    teams = overall_strength_to_TOI[year].keys()
    for _team in teams:
        curr_row = [year, _team]
        for _strength in strengths:
            curr_toi = overall_strength_to_TOI[year][_team][_strength]
            curr_goals = overall_strength_to_goals[year][_team][_strength]
            curr_goals_against = overall_strength_to_goals_against[year][_team][_strength]
            new_row = curr_row + [_strength, curr_toi, curr_goals, curr_goals_against]
            data.append(new_row)
            
df = pd.DataFrame(data, columns=columns)
df.to_csv("./nhl_playoff_strength_data.csv", encoding="latin-1")

/var/folders/sy/pk0rd4q97v1b5s6drdrrz6q00000gn/T/ipykernel_20824/2183932385.py:57: DtypeWarning: Columns (56,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  curr_pbp = pd.read_csv(file, encoding='latin-1')
410520it [00:20, 20236.49it/s]


Done


/var/folders/sy/pk0rd4q97v1b5s6drdrrz6q00000gn/T/ipykernel_20824/2183932385.py:57: DtypeWarning: Columns (47,54,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  curr_pbp = pd.read_csv(file, encoding='latin-1')
425270it [00:22, 12876.53it/s]

PIM difference == 7
San Jose Sharks @ Vegas Golden Knights 2018-04-26
(300, 120) ()


429082it [00:22, 12207.45it/s]

...

PIM difference == 9
Assuming a 5+2+2
Vegas Golden Knights @ Colorado Avalanche 2021-05-30
(540,) ()


284536it [00:14, 19661.04it/s]


Done


/var/folders/sy/pk0rd4q97v1b5s6drdrrz6q00000gn/T/ipykernel_20824/2183932385.py:57: DtypeWarning: Columns (49,56) have mixed types. Specify dtype option on import or set low_memory=False.
  curr_pbp = pd.read_csv(file, encoding='latin-1')
431914it [00:21, 19715.08it/s]


Done


In [22]:
def scrape_pbp_debug():
    team_to_power_plays = {}
    current_game = None
    penalty_just_called = False

    # these are irrelevant to on-ice manpower and PP goals (misconducts usually paired with a minor/major)
    irrelevant_penalties = set(['Penalty Shot', 'Misconduct', 'Game Misconduct'])
    
    # debugging variables
    i = 0
    game_num = 118
    debug = False

    for _, play in tqdm(pbp_18_19.iterrows()):
        event = play['event_type']

        away_team, home_team, date = play['away_name'], play['home_name'], play['game_date']
        if current_game:
            if away_team != current_game.away_team or home_team != current_game.home_team or date != current_game.date:
                current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])

        if penalty_just_called and event != 'PENALTY': # once all penalties are announced, process them and figure out the manpower
            current_game.process_penalties(play, debug)
            penalty_just_called = False

        if play['period'] == 4 and current_game.season_type == 'R':
            if not current_game.regular_season_OT:
                current_game.start_OT()
            elif play['event_type'] in set(['STOP', 'CHALLENGE', 'PENALTY']):
                current_game.reset_OT_strengths()

        match event:
            case 'GAME_SCHEDULED':
                current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])
#                 if current_game.away_team == 'Columbus Blue Jackets' and current_game.home_team == 'Boston Bruins':
#                     print(f'bos @ cbj i={i}')
                if current_game.away_team == 'Chicago Blackhawks' and current_game.home_team == 'Arizona Coyotes':
                    print(f'chi @ ari i={i}')
#                 if i == game_num: # debugging
#                     print(f'New Game: {current_game.away_team} @ {current_game.home_team}')
            case 'PENALTY':
                if play['penalty_severity'] not in irrelevant_penalties:  # minor/major/misconduct/game misconduct/match
                    current_game.add_penalty(play)
                    penalty_just_called = True
            case 'GOAL': # exclude penalty shots and shootout goals
                if play['secondary_type'] != 'Penalty Shot' and play['period'] != 5: # exclude SO
                    current_game.add_goal(play, debug)
            case 'GAME_END':
                # debugging 
                i += 1
                if i > game_num - 1:
                    debug = True
                if i > game_num:
                    break # debugging
            case default:
                current_game.update_penalty_clock(play['period'], play['period_seconds'], debug)

    print('Done')

     

In [23]:
scrape_pbp_debug()

25848it [00:03, 13748.42it/s]

multiple expiring penalties: Calgary Flames @ Vancouver Canucks 2017-10-14


38200it [00:04, 9089.55it/s] 


chi @ ari i=118
defaultdict(<class 'int'>, {(5, 5): 0})
Updating Penalty Clock at P:1 0s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 31})
Updating Penalty Clock at P:1 31s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 35})
Updating Penalty Clock at P:1 35s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 41})
Updating Penalty Clock at P:1 41s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 61})
Updating Penalty Clock at P:1 61s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 61})
Updating Penalty Clock at P:1 61s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 65})
Updating Penalty Clock at P:1 65s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 65})
Updating Penalty Clock at P:1 65s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 77})
Updating Penalty Clock at P:1 77s
Away: []
Home: []
(5, 5)
defaultdict(<class 'int'>, {(5, 5): 80})
Updating Penalty Clock at P:1 80s
Away: []
Ho

In [ ]:
def get_coincidental_penalties():
    ### Below is a debugging function to check for different coincidental penalty situations in a given season
    current_game = None
    penalty_just_called = False
    coincidentals = Counter()

    # these are irrelevant to on-ice manpower and PP goals (misconducts usually paired with a minor/major)
    irrelevant_penalties = set(['Penalty Shot', 'Misconduct', 'Game Misconduct'])

    i = 0
    pbp_test = ''
    for year in range(16, 23):
        pbp_test = pd.read_csv(f"./play_by_play_20{year}_{year+1}_lite.csv", encoding='latin-1')
        for _, play in tqdm(pbp_test.iterrows()):
            event = play['event_type']

            away_team, home_team, date = play['away_name'], play['home_name'], play['game_date']
            if current_game:
                if away_team != current_game.away_team or home_team != current_game.home_team or date != current_game.date:
                    current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])

            match event:
                case 'GAME_SCHEDULED':
                    current_game = Game(play['away_name'], play['home_name'], play['game_date'], play['season_type'])
                case 'PENALTY':
                    if play['penalty_severity'] not in irrelevant_penalties:  # minor/major/misconduct/game misconduct/match
                        current_game.add_penalty_debug(play)
                        penalty_just_called = True
                case 'GAME_END':
                    coincidentals += current_game.coincidentals

            if penalty_just_called and event != 'PENALTY': # once all penalties are announced, process them and figure out the manpower
                current_game.process_penalties_debug()
                penalty_just_called = False

        for k, _ in coincidentals.items():
            t1, t2 = k
            abs_PIM_diff = abs(sum(t1) - sum(t2))
            coincidental_penalty_cache[abs_PIM_diff].append((t1, t2))

        coincidentals = Counter()

In [99]:
pbp_18_19[320:340]

,xg,event_id,event_type,event,secondary_type,event_team,event_team_type,description,period,period_seconds,...,home_division_name,home_division_name_short,home_conference_name,home_id,away_name,away_abbreviation,away_division_name,away_division_name_short,away_conference_name,away_id
320,NaN,2.021020e+13,FACEOFF,Faceoff,NaN,Seattle Kraken,away,Jared McCann faceoff won against Chandler Step...,1,0,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
321,0.017389,2.021020e+13,SHOT,Shot,Slap Shot,Seattle Kraken,away,Mark Giordano Slap Shot saved by Robin Lehner,1,18,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
322,NaN,2.021020e+13,BLOCKED_SHOT,Blocked Shot,NaN,Seattle Kraken,away,Jaden Schwartz shot blocked shot by Chandler S...,1,20,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
323,0.002270,2.021020e+13,MISSED_SHOT,Missed Shot,NaN,Seattle Kraken,away,Jordan Eberle Wide of Net Robin Lehner,1,26,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
324,NaN,2.021020e+13,PENALTY,Penalty,Tripping,Vegas Golden Knights,home,Alex Pietrangelo Tripping against Jaden Schwartz,1,27,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
325,NaN,2.021020e+13,FACEOFF,Faceoff,NaN,Vegas Golden Knights,home,William Karlsson faceoff won against Morgan Ge...,1,27,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
326,NaN,2.021020e+13,HIT,Hit,NaN,Vegas Golden Knights,home,Brayden McNabb hit Marcus Johansson,1,50,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
327,NaN,2.021020e+13,GIVEAWAY,Giveaway,NaN,Vegas Golden Knights,home,Giveaway by William Karlsson,1,56,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
328,0.060751,2.021020e+13,SHOT,Shot,Wrist Shot,Seattle Kraken,away,Morgan Geekie Wrist Shot saved by Robin Lehner,1,58,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55
329,NaN,2.021020e+13,BLOCKED_SHOT,Blocked Shot,NaN,Seattle Kraken,away,Vince Dunn shot blocked shot by Brayden McNabb,1,80,...,Pacific,PAC,Western,54,Seattle Kraken,SEA,Pacific,PAC,Western,55


In [46]:
pbp = pd.read_csv("./play_by_play_2022_23_lite.csv")
pbp.loc[pbp['away_abbreviation'] == 'MTL']

/var/folders/sy/pk0rd4q97v1b5s6drdrrz6q00000gn/T/ipykernel_20824/3467332724.py:1: DtypeWarning: Columns (49,56) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp = pd.read_csv("./play_by_play_2022_23_lite.csv", encoding="latin-1")


,xg,event_id,event_type,event,secondary_type,event_team,event_team_type,description,period,period_seconds,...,home_division_name_short,home_conference_name,home_id,away_name,away_abbreviation,away_division_name,away_division_name_short,away_conference_name,away_id,venue_id
6824,NaN,2.022020e+13,GAME_SCHEDULED,Game Scheduled,NaN,NaN,NaN,Game Scheduled,1,0,...,ATL,Eastern,17,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5145.0
6825,NaN,2.022020e+13,FACEOFF,Faceoff,NaN,Detroit Red Wings,home,Dylan Larkin faceoff won against Nick Suzuki,1,0,...,ATL,Eastern,17,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5145.0
6826,NaN,2.022020e+13,HIT,Hit,NaN,Detroit Red Wings,home,Dylan Larkin hit Nick Suzuki,1,10,...,ATL,Eastern,17,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5145.0
6827,NaN,2.022020e+13,GIVEAWAY,Giveaway,NaN,Detroit Red Wings,home,Giveaway by Dylan Larkin,1,27,...,ATL,Eastern,17,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5145.0
6828,0.047776,2.022020e+13,SHOT,Shot,Slap Shot,MontrÃ©al Canadiens,away,Cole Caufield Slap Shot saved by Ville Husso,1,33,...,ATL,Eastern,17,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239750,0.213533,2.022021e+13,GOAL,Goal,Wrist Shot,Ottawa Senators,home,"Derick Brassard (8) Wrist Shot, assists: Drake...",3,1152,...,ATL,Eastern,9,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5031.0
239751,NaN,2.022021e+13,FACEOFF,Faceoff,NaN,Ottawa Senators,home,Dylan Gambrell faceoff won against Christian D...,3,1152,...,ATL,Eastern,9,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5031.0
239752,NaN,2.022021e+13,HIT,Hit,NaN,MontrÃ©al Canadiens,away,Jesse Ylonen hit Nick Holden,3,1168,...,ATL,Eastern,9,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5031.0
239753,NaN,2.022021e+13,PERIOD_END,Period End,NaN,NaN,NaN,End of 3rd Period,3,1200,...,ATL,Eastern,9,MontrÃ©al Canadiens,MTL,Atlantic,ATL,Eastern,8,5031.0


In [45]:
pbp_21 = pd.read_csv("./play_by_play_2021_22_lite.csv")
pbp.loc[pbp['away_abbreviation'] == 'MTL']

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 17245: invalid continuation byte